In [ ]:
import pandas as pd

df = pd.read_csv("15MIN-Updated.csv", parse_dates=["DateTime"])
df.set_index("DateTime", inplace=True)

# Optional: create CrossSignal, SL, TP columns if not already present
# df['CrossSignal'] = ...
# df['SL'] = ...
# df['TP'] = ...
import backtrader as bt

class CSVData(bt.feeds.PandasData):
    lines = ('CrossSignal', 'SL', 'TP')
    params = (
        ('datetime', None),
        ('open', 'Open'),
        ('high', 'High'),
        ('low', 'Low'),
        ('close', 'Close'),
        ('volume', -1),
        ('openinterest', -1),
        ('CrossSignal', -1),
        ('SL', -1),
        ('TP', -1),
    )

data = CSVData(dataname=df)


In [ ]:
import logging
from typing import List, Dict, Any, Optional, Union
import backtrader as bt
import pandas as pd
from datetime import datetime

# Configure logging
logger = logging.getLogger(__name__)

class TradeLogger:
    """Manages trade logging and storage for trading strategies."""

    def __init__(self):
        self.trades: List[Dict[str, Any]] = []

    def log_trade(
        self,
        trade_id: int,
        symbol: str,
        strategy_name: str,
        entry_time: datetime,
        exit_time: datetime,
        entry_price: float,
        exit_price: float,
        position_size: int,
        direction: str,
        pnl: float,
        entry_index: int,
        exit_index: int,
        stop_loss: Optional[float],
        take_profit: Optional[float],
        commission: float,
        spread: float
    ) -> None:
        """
        Log a single trade with all relevant details.

        Args:
            trade_id: Unique identifier for the trade.
            symbol: Trading instrument symbol.
            strategy_name: Name of the strategy.
            entry_time: Datetime of trade entry.
            exit_time: Datetime of trade exit.
            entry_price: Price at trade entry.
            exit_price: Price at trade exit.
            position_size: Size of the position.
            direction: Trade direction ("Long" or "Short").
            pnl: Profit and loss of the trade.
            entry_index: Candle index at entry.
            exit_index: Candle index at exit.
            stop_loss: Stop-loss price.
            take_profit: Take-profit price.
            commission: Commission paid for the trade.
            spread: Spread at the time of trade.
        """
        trade_data = {
            "Trade ID": trade_id,
            "Symbol": symbol,
            "Strategy Name": strategy_name,
            "Entry Time": entry_time,
            "Exit Time": exit_time,
            "Entry Price": entry_price,
            "Exit Price": exit_price,
            "Position Size": position_size,
            "Direction": direction,
            "PnL": pnl,
            "PnL %": (pnl / entry_price * 100) if entry_price else None,
            "Candle Index Entry": entry_index,
            "Candle Index Exit": exit_index,
            "SL": stop_loss,
            "TP": take_profit,
            "Risk-Reward Ratio": (
                (take_profit - entry_price) / (entry_price - stop_loss)
                if stop_loss is not None and take_profit is not None and entry_price > stop_loss
                else None
            ),
            "Commissions": commission,
            "Spread": spread,
        }
        self.trades.append(trade_data)
        logger.debug(f"Logged trade: {trade_data}")

    def to_dataframe(self) -> pd.DataFrame:
        """
        Convert logged trades to a pandas DataFrame.

        Returns:
            DataFrame containing all trade records.
        """
        return pd.DataFrame(self.trades)

class CrossSignalStrategy(bt.Strategy):
    """
    A Backtrader strategy that executes trades based on a cross signal with stop-loss
    and take-profit levels.
    """
    params = (
        ("size", 1),  # Default position size (1 lot)
        ("valid_signal_values", {1}),  # Valid signal values for long trades
    )

    def __init__(self):
        """Initialize the strategy with data bindings and trade logger."""
        self.cross_signal = self.datas[0].CrossSignal
        self.sl = self.datas[0].SL
        self.tp = self.datas[0].TP
        self.trade_logger = TradeLogger()
        self.last_entry_price: Optional[float] = None
        self.last_entry_datetime: Optional[datetime] = None
        self.last_entry_sl: Optional[float] = None
        self.last_entry_tp: Optional[float] = None
        self.last_entry_index: Optional[int] = None
        logger.info("Initialized CrossSignalStrategy")

    def next(self) -> None:
        """
        Execute trading logic for each new bar.

        Validates signals and places buy orders with stop-loss and take-profit.
        """
        if self.position:
            return  # Skip if already in a position

        current_signal = self.cross_signal[0]
        if current_signal not in self.params.valid_signal_values:
            return  # Invalid or no signal

        # Validate SL and TP
        current_sl = self.sl[0]
        current_tp = self.tp[0]
        current_price = self.data.open[0]

        if not self._validate_trade_parameters(current_price, current_sl, current_tp):
            logger.warning(
                f"Invalid trade parameters: price={current_price}, SL={current_sl}, TP={current_tp}"
            )
            return

        # Store entry details
        self.last_entry_sl = current_sl
        self.last_entry_tp = current_tp
        self.last_entry_price = current_price
        self.last_entry_datetime = self.data.datetime.datetime(0)
        self.last_entry_index = len(self) - 1

        # Place buy bracket order
        try:
            self.buy_bracket(
                size=self.params.size,
                price=current_price,
                stopprice=current_sl,
                limitprice=current_tp
            )
            logger.info(
                f"Placed buy order: price={current_price}, SL={current_sl}, TP={current_tp}"
            )
        except Exception as e:
            logger.error(f"Failed to place order: {e}")
            raise

    def notify_order(self, order: bt.Order) -> None:
        """
        Handle order notifications and update entry details.

        Args:
            order: The Backtrader order object.
        """
        if order.status == order.Completed and order.isbuy():
            self.last_entry_price = order.executed.price
            self.last_entry_datetime = self.data.datetime.datetime(0)
            self.last_entry_index = len(self) - 1
            logger.debug(f"Order completed: Buy at {self.last_entry_price}")

    def notify_trade(self, trade: bt.Trade) -> None:
        """
        Log trade details when a trade is closed.

        Args:
            trade: The Backtrader trade object.
        """
        if not trade.isclosed:
            return

        try:
            self.trade_logger.log_trade(
                trade_id=len(self.trade_logger.trades) + 1,
                symbol=self.data._name,
                strategy_name=type(self).__name__,
                entry_time=self.last_entry_datetime,
                exit_time=self.data.datetime.datetime(0),
                entry_price=self.last_entry_price,
                exit_price=trade.price,
                position_size=trade.size,
                direction="Long",
                pnl=trade.pnl,
                entry_index=self.last_entry_index,
                exit_index=len(self) - 1,
                stop_loss=self.last_entry_sl,
                take_profit=self.last_entry_tp,
                commission=self.broker.getcommissioninfo(self.data).getcommission(
                    trade.size, trade.price
                ),
                spread=self.data.high[0] - self.data.low[0],
            )
        except Exception as e:
            logger.error(f"Error logging trade: {e}")

    def stop(self) -> None:
        """Finalize the strategy and output trade summary."""
        trades_df = self.trade_logger.to_dataframe()
        if trades_df.empty:
            logger.info("No trades executed.")
        else:
            logger.info(f"Total trades executed: {len(trades_df)}")
            # Optionally save to CSV in production
            # trades_df.to_csv("trades_log.csv", index=False)

    def _validate_trade_parameters(
        self, price: float, stop_loss: float, take_profit: float
    ) -> bool:
        """
        Validate trade parameters (price, SL, TP).

        Args:
            price: Entry price.
            stop_loss: Stop-loss price.
            take_profit: Take-profit price.

        Returns:
            True if parameters are valid, False otherwise.
        """
        if not all(isinstance(x, (int, float)) for x in [price, stop_loss, take_profit]):
            return False
        if price <= stop_loss or price >= take_profit:
            return False
        return True

In [65]:
cerebro = bt.Cerebro()
cerebro.adddata(data)
cerebro.addstrategy(CrossSignalStrategy, size=1, valid_signal_values={1})
cerebro.broker.setcash(100000)  # Starting capital
cerebro.broker.setcommission(commission=0.001)


In [66]:
results = cerebro.run()
# Access the strategy instance
strat = results[0]

# Convert trades to DataFrame
trades_df = strat.trade_logger.to_dataframe()
trades_df.iloc[1].T

# Optional: save to CSV
# trades_df.to_csv("trades_log.csv", index=False)


INFO:__main__:Initialized CrossSignalStrategy


AttributeError: 'Lines_LineSeries_DataSeries_OHLC_OHLCDateTime_AbstractDataBase_DataBase_PandasData_CSVData' object has no attribute 'clsoe'

In [ ]:
journal.iloc[1].T